<a href="https://colab.research.google.com/github/Loop3D/2022_iamg_workshop/blob/main/draw_your_own.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing dependencies
This notebook will use two libraries from the Loop project
1. LoopStructural - https://github.com/Loop3D/LoopStructural
2. map2loop - https://github.com/Loop3D/map2loop-2


The following code blocks will install the required dependencies for the Loop libraries into this colab environment. Alternatively, the workshop can be run using the docker image.

## map2loop + LoopStructural

In [ ]:
!pip install rasterio
!pip install git+https://github.com/geopandas/geopandas.git@v0.10.2
!pip install hjson
!pip install owslib
!pip install git+https://github.com/Loop3D/map2model_cpp.git
!pip install git+https://github.com/Loop3D/LoopProjectFile.git
!pip install pygeos
!pip install mplstereonet
!pip install lavavu-osmesa
!pip install git+https://github.com/Loop3d/LoopStructural
!pip install git+https://github.com/Loop3D/map2loop-2.git
!pip install beartype
!pip install git+https://github.com/giswqs/leafmap.git


In [ ]:
#import ipywidgets as widgets
import os


# load last saved map area and model engine (if they exist)
if(not os.path.isdir('../scratch/')):
    os.mkdir('../scratch/')
test_data_name='Draw Your Own'

In [ ]:
import leafmap
import pandas as pd
import json
import random
from shapely.geometry import Polygon

import ipywidgets as widgets
import geopandas as gpd
from shapely.geometry import Polygon


if(not test_data_name =='Draw Your Own'):
    if(test_data_name=='Last Area Drawn'):
        last_coords=pd.read_csv('../scratch/last_area.csv')
        display(last_coords)
        minx=last_coords.iloc[0]['minx']
        miny=last_coords.iloc[0]['miny']
        maxx=last_coords.iloc[0]['maxx']
        maxy=last_coords.iloc[0]['maxy']
    elif(not test_data_name =='Draw Your Own'):
        y_point_list = [miny, miny, maxy, maxy, maxy]
        x_point_list = [minx, maxx, maxx, minx, minx]
        bbox_geom = Polygon(zip(x_point_list, y_point_list))
        polygon = gpd.GeoDataFrame(index=[0], crs=dst_crs, geometry=[bbox_geom])
        polygon_ll=polygon.to_crs(src_crs)

        minx=polygon_ll.total_bounds[0]
        maxx=polygon_ll.total_bounds[2]
        miny=polygon_ll.total_bounds[1]
        maxy=polygon_ll.total_bounds[3]

        minlong=minx
        maxlong=maxx
        minlat=miny
        maxlat=maxy
        #print("x",polygon_ll.total_bounds[0])
        st_bbox=[minlong,minlat,maxlong,maxlat]
        lat_point_list = [minlat, minlat, maxlat, maxlat,maxlat]
        lon_point_list = [minlong, maxlong, maxlong, minlong, minlong]
        bbox_geom = Polygon(zip(lon_point_list, lat_point_list))
        rect = gpd.GeoDataFrame(index=[0], crs=src_crs, geometry=[bbox_geom]) 

    
    src_crs = "epsg:4326"  # coordinate reference system for imported dtms (geodetic lat/long WGS84)
    dst_crs = "epsg:28350"  # coordinate reference system for imported dtms (geodetic lat/long WGS84)

    bbox2=str(minx)+","+str(miny)+","+str(maxx)+","+str(maxy)
    y_point_list = [miny, miny, maxy, maxy, maxy]
    x_point_list = [minx, maxx, maxx, minx, minx]
    bbox_geom = Polygon(zip(x_point_list, y_point_list))
    polygon = gpd.GeoDataFrame(index=[0], crs=dst_crs, geometry=[bbox_geom])
    polygon_ll=polygon.to_crs(src_crs)

    minlong=polygon_ll.total_bounds[0]
    maxlong=polygon_ll.total_bounds[2]
    minlat=polygon_ll.total_bounds[1]
    maxlat=polygon_ll.total_bounds[3]
    
    minlong=minx
    maxlong=maxx
    minlat=miny
    maxlat=maxy

    lat_point_list = [minlat, minlat, maxlat, maxlat,maxlat]
    lon_point_list = [minlong, maxlong, maxlong, minlong, minlong]
    bbox_geom = Polygon(zip(lon_point_list, lat_point_list))
    rect = gpd.GeoDataFrame(index=[0], crs=src_crs, geometry=[bbox_geom]) 

    example_rect = GeoData(geo_dataframe = rect,
                   style={'color': 'purple', 'opacity':3, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},                  
                   name = 'Example')
    center=(minlat+((maxlat-minlat)/2),minlong+((maxlong-minlong)/2))
else:
    center=(-22.6,117.3)

    
m =leafmap.Map( center=center, zoom=8,scroll_wheel_zoom=True)
m.add_basemap( basemap='OpenTopoMap')
#label = leafmap.Label()
#display(label)

#def handle_interaction(**kwargs):
#    if kwargs.get('type') == 'mousemove':
#        label.value = str(kwargs.get('coordinates'))
#m.on_interaction(handle_interaction)

m.add_wms_layer(url='https://www.loopwms.xyz/geoserver/loop/wms?',
    layers='2_5m_interpgeop15_4326',
    format='image/png',
    transparent=True,
    opacity=0.4,
    attribution='Geology data from GSWA',
    name='geology',
    shown=True)
#aurl='https://www.loopwms.xyz/geoserver/loop/wms?'
m.add_wms_layer(url='https://www.loopwms.xyz/geoserver/loop/wms?',
#m.add_wms_layer(url='http://13.211.217.129:8080/geoserver/loop/wms?',
    layers='2_5m_interpstrucl15_4326',
#    layers='0',
    format='image/png',
    transparent=True,
    opacity=0.4,
    attribution='Linear features data from GSWA',
    name='faults/folds')

m.add_wms_layer(url='https://www.loopwms.xyz/geoserver/loop/wms?',
    layers='waroxi_wa_4326_bed',
    format='image/png',
    transparent=True,
    attribution='Outcrop data from GSWA',
    name='outcrops')
if( not test_data_name =='Draw Your Own'):
    m.add_layer(example_rect)

    


#m.add_control(LayersControl())
#m.add_control(FullScreenControl())
#m.add_control(ScaleControl(position='topright'))

#dc = DrawControl(rectangle={'shapeOptions': {'color': '#0000FF'}})
#m.add_control(dc)


m

In [ ]:

rect=m.draw_features
if(len(rect)>0):
  print(rect[0]['geometry']['coordinates'])
else:
  print('Default rectangle used, as none drawn')

In [ ]:


src_crs = "epsg:4326"  # coordinate reference system for imported dtms (geodetic lat/long WGS84)
dst_crs = "epsg:28350" # coordinate system for example data


use_roi_clip=False
roi_clip_path=''
    
if(len(rect)==0):
  minlong=117.13698
  maxlong=117.564464
  minlat= -22.690712
  maxlat=-22.396454

else:
  minlong=rect[0]['geometry']['coordinates'][0][0][0]
  maxlong=rect[0]['geometry']['coordinates'][0][2][0]
  minlat= rect[0]['geometry']['coordinates'][0][0][1]
  maxlat= rect[0]['geometry']['coordinates'][0][1][1]


bounds=(minlong,maxlong,minlat,maxlat)


lat_point_list = [minlat, minlat, maxlat, maxlat,maxlat]
lon_point_list = [minlong, maxlong, maxlong, minlong, minlong]
bbox_geom = Polygon(zip(lon_point_list, lat_point_list))
mbbox = gpd.GeoDataFrame(index=[0], crs=src_crs, geometry=[bbox_geom]) 
bbox=mbbox.total_bounds
st_bbox=[bbox[0],bbox[1],bbox[2],bbox[3]]
print(src_crs,mbbox.total_bounds)
mbbox=mbbox.to_crs(dst_crs)
print(dst_crs,mbbox.total_bounds)

In [ ]:
import os
import hjson
from map2loop.m2l_enums import VerboseLevel
from map2loop.project import Project

savepath='./newnew3'
proj = Project( 
                 loopdata_state = "WA",
            project_path=savepath,
    project_crs='EPSG:28350',
    verbose_level=VerboseLevel.NONE,
    working_projection='EPSG:28350',
                )

proj.update_config(
                    out_dir=savepath,
                    bbox_3d={
                         "minx": mbbox.total_bounds[0], #500000,
                         "miny": mbbox.total_bounds[1], #7490000,
                         "maxx": mbbox.total_bounds[2], #545000,
                         "maxy": mbbox.total_bounds[3], #7520000,
                         "base": -4800,
                         "top": 1200,
                    },
                    project_crs='EPSG:28350',
                    # overwrite='true',
                    # run_flags={'fault_dip':-999},
                    # quiet='None',
    
#                     loopFilename='test.loop3d'
                  )


In [ ]:
proj.run()

In [ ]:
minx,miny,maxx,maxy = proj.config.bbox
model_base = proj.config.bbox_3d['base']
model_top = proj.config.bbox_3d['top']

In [ ]:
f=open(proj.config.tmp_path+'/bbox.csv','w')
f.write('minx,miny,maxx,maxy,lower,upper\n')
ostr='{},{},{},{},{},{}\n'.format(minx,miny,maxx,maxy,model_base,model_top)
f.write(ostr)
f.close()

In [ ]:
from LoopStructural import GeologicalModel
fault_params = {'interpolatortype':'FDI',
                'nelements':1e4,
                'step':10,
                'fault_buffer':0.2,
#                 'force_mesh_geometry':True,
#                 'solver':'pyamg',
#                 overprints:overprints,
#                 'cpw':1,
#                 'gpw':5,
               }
foliation_params = {'interpolatortype':'FDI' , # 'interpolatortype':'PLI',
                    'nelements':5e4,  # how many tetras/voxels
                    'buffer':2.,  # how much to extend nterpolation around box
#                     'solver':'pyamg',
#                     'damp':True
                    # 'npw':0,
                    # 'regularisation':0.5
                   }


model, m2l_data = GeologicalModel.from_map2loop_directory(proj.config.project_path,
                                                              evaluate=False,
                                                          fault_params=fault_params,
                                                          rescale=False,
#                                                           vectorscale=1,
                                                          foliation_params=foliation_params)

In [ ]:
model.update()

In [ ]:
from LoopStructural.visualisation import LavaVuModelViewer

In [ ]:
view = LavaVuModelViewer(model)
view.add_model_surfaces()
view.interactive()

In [ ]:
view.display()